In [ ]:
#批量修改节点名称

In [ ]:
def read_inp_sections_with_encoding(file_path, encoding='latin1'):
    sections = {}
    current_section = None
    
    with open(file_path, 'r', encoding=encoding) as file:
        for line in file:
            if line.startswith('[') and line.endswith(']\n'):
                current_section = line.strip()
                sections[current_section] = []
            elif current_section is not None:
                line = line.strip()
                if line and not line.startswith(';'):
                    sections[current_section].append(line)
    
    return sections

def prefix_all_node_names_corrected(sections, prefix='J'):
    # Sections where direct node names are mentioned
    node_sections = ['[JUNCTIONS]', '[OUTFALLS]', '[COORDINATES]', '[VERTICES]']
    
    # Update node names in direct sections
    for section in node_sections:
        if section in sections:
            modified_section = []
            for line in sections[section]:
                parts = line.split()
                if not parts[0].startswith(prefix):
                    parts[0] = prefix + parts[0]
                modified_section.append(' '.join(parts))
            sections[section] = modified_section

    # Correctly modify the outlet names in [SUBCATCHMENTS]
    if '[SUBCATCHMENTS]' in sections:
        modified_subcatchments = []
        for line in sections['[SUBCATCHMENTS]']:
            parts = line.split()
            if not parts[2].startswith(prefix):
                parts[2] = prefix + parts[2]
            modified_subcatchments.append(' '.join(parts))
        sections['[SUBCATCHMENTS]'] = modified_subcatchments

    # Update start and end nodes in [CONDUITS]
    if '[CONDUITS]' in sections:
        modified_conduits = []
        for line in sections['[CONDUITS]']:
            parts = line.split()
            if not parts[1].startswith(prefix):
                parts[1] = prefix + parts[1]
            if not parts[2].startswith(prefix):
                parts[2] = prefix + parts[2]
            modified_conduits.append(' '.join(parts))
        sections['[CONDUITS]'] = modified_conduits

    return sections

def write_modified_inp(file_path, sections):
    with open(file_path, 'w', encoding='latin1') as file:
        for section, lines in sections.items():
            file.write(section + '\n')
            for line in lines:
                file.write(line + '\n')
            file.write('\n')  # Ensure good section separation

# Specify your original and destination file paths
inp_file_path = './inpFile.inp'  # Update this path
corrected_modified_inp_file_path = './corrected_fully_modifiedFile.inp'  # Update this path

# Process
sections_encoded = read_inp_sections_with_encoding(inp_file_path)
sections_fully_corrected = prefix_all_node_names_corrected(sections_encoded)
write_modified_inp(corrected_modified_inp_file_path, sections_fully_corrected)


In [ ]:
from pyswmm import Simulation, Nodes
import numpy as np
import re

def parse_coordinates(inp_file):
    """ 从 SWMM .inp 文件中解析节点坐标 """
    coords = {}
    with open(inp_file, 'r') as file:
        read_coords = False
        for line in file:
            if line.startswith('[COORDINATES]'):
                read_coords = True
                continue
            if read_coords:
                if line.startswith('['):  # 结束坐标部分
                    break
                parts = line.split()
                if len(parts) >= 3:
                    try:
                        node_id, x, y = parts[0], float(parts[1]), float(parts[2])
                        coords[node_id] = (x, y)
                    except ValueError:
                        continue
    return coords

def run_simulation_and_collect_data(inp_file):
    """ 运行模拟并收集节点的溢流数据 """
    node_coords = parse_coordinates(inp_file)

    time_steps = np.arange(0, 180 + 1, 5) * 60  # 转换为秒
    node_data = {node_id: {'coords': coords, 'overflows': [0]*len(time_steps)} for node_id, coords in node_coords.items()}  # 初始化溢流数组以匹配时间步数

    # 运行模拟并收集数据
    with Simulation(inp_file) as sim:
        time_step_index = 0  # 跟踪当前时间步长的索引
        for step in sim:
            current_time = sim.current_time
            elapsed_time = (current_time - sim.start_time).total_seconds()
            # 更新数据直到当前步骤的时间超过下一个预定时间步长
            while time_step_index < len(time_steps) and elapsed_time >= time_steps[time_step_index]:
                for node_id in node_coords.keys():
                    swmm_node = Nodes(sim)[node_id]
                    # 更新溢流数据
                    node_data[node_id]['overflows'][time_step_index] = swmm_node.flooding
                time_step_index += 1

    # 筛选出有溢流记录的节点
    overflow_nodes = {node_id: data for node_id, data in node_data.items() if any(data['overflows'])}

    return overflow_nodes, time_steps.tolist(), node_coords


def write_results_to_file(overflow_results, node_coords, time_steps, file_name):
    """ 将结果写入到文本文件中 """
    with open(file_name, 'w') as file:
        file.write("Time Steps (in seconds): {}\n".format(time_steps))
        file.write("Overflow Nodes:\n")
        for node_id, data in overflow_results.items():
            file.write("{}: Overflows: {}\n".format(node_id, data['overflows']))

        if overflow_results:
            file.write("\nNode Coordinates:\n")
            for node_id in overflow_results.keys():
                file.write("{}: Coordinates: {}\n".format(node_id, node_coords[node_id]))

# 调用函数并传入你的 SWMM 模型文件路径
inp_file = './inpFile.inp'  # 替换为您的模型文件路径
overflow_results, time_steps, node_coords = run_simulation_and_collect_data(inp_file)

# 将结果写入文件
if overflow_results:
    file_name = 'overflow_results_test.txt'
    write_results_to_file(overflow_results, node_coords, time_steps, file_name)
    print(f"Results have been written to {file_name}")
else:
    print("No overflows occurred.")

In [ ]:
import re

# Function to parse overflow results from the content string
def parse_overflow_results(content):
    # Find all node overflow entries
    overflow_entries = re.findall(r'(J\d+): Overflows: \[([0-9.,\s]+)\]', content)
    
    # Initialize dictionary to store parsed results
    parsed_results = {}
    
    # Process each entry
    for node, overflows_str in overflow_entries:
        # Convert the string of overflows into a list of floats
        overflows = [float(x)/float(17*17) for x in overflows_str.split(', ')]
        parsed_results[node] = overflows
    
    return parsed_results

# Function to write formatted data to a new file with spacing between nodes
def write_formatted_data_with_spacing(filepath, nodes, time_steps):
    with open(filepath, 'w') as file:
        # Write the header
        file.write('points                                  \n')
        
        # Loop through each node to write its data with an extra newline for spacing
        for node, overflows in nodes.items():
            # Ensure the number of overflows match the time steps, if not append 0's to make them match
            if len(overflows) < len(time_steps):
                overflows.extend([0.0] * (len(time_steps) - len(overflows)))
            
            file.write(f'{node}                                  \n')  # Node name
            file.write(f'{len(time_steps)}                                seconds\n')  # Number of time steps
            for time_step, overflow in zip(time_steps, overflows):
                file.write(f'                 {overflow}                 {time_step}\n')  # Overflow value and time step
            file.write('\n')  # Newline for spacing between nodes

# Read the content of the newly uploaded overflow results file
file_path_new = './overflow_results_test.txt'
with open(file_path_new, 'r') as file:
    new_overflow_content = file.read()

# Extract the time steps from the new content
time_steps_match_new = re.search(r'Time Steps \(in seconds\): \[([0-9.,\s]+)\]', new_overflow_content)
time_steps_new = [int(x) for x in time_steps_match_new.group(1).split(', ')] if time_steps_match_new else []

# Parse the new overflow content to get a structured representation
parsed_overflow_nodes_new = parse_overflow_results(new_overflow_content)

# Generate a new file path for the updated data based on the new file
new_file_path_updated_new = './overflow_results_updated_new_formatted.bdy'

# Write the formatted data to the new file with the correct time steps and spacing based on the new file
write_formatted_data_with_spacing(new_file_path_updated_new, parsed_overflow_nodes_new, time_steps_new)

# Output the path for verification
print(f'Formatted file created at: {new_file_path_updated_new}')
